In [1]:
import os
import sys
import json
import time
import gspread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Patch
from matplotlib.gridspec import GridSpec
from scipy import integrate, signal, fftpack, stats, ndimage
import statsmodels.stats.multitest as multitest

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import (
    get_stim_events,
    get_evoked_traces,
    get_evoked_firing_rates,
    find_nearest_ind
)
from allensdk.brain_observatory.ecephys.lfp_subsampling.subsampling import subsample_lfp, remove_lfp_offset
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [4]:
%matplotlib notebook

Load CCF for identifying cortical areas

In [5]:
mcc = MouseConnectivityCache(resolution=10)
str_tree = mcc.get_structure_tree()

Load Zap_Zip-log_exp to get metadata for experiments

In [6]:
_gc = gspread.service_account() # need a key file to access the account
_sh = _gc.open('Zap_Zip-log_exp') # open the spreadsheet
_df = pd.DataFrame(_sh.sheet1.get()) # load the first worksheet
zzmetadata = _df.T.set_index(0).T # put it in a nicely formatted dataframe

Define areas of interest to plot population activity

In [7]:
areas_of_interest = {
    'MO': [
        'MOp1', 'MOp2/3', 'MOp5', 'MOp6a', 'MOp6b',
        'MOs1', 'MOs2/3', 'MOs5', 'MOs6a', 'MOs6b'
    ],
    'ACA': [
        'ACAd1', 'ACAd2/3', 'ACAd5', 'ACAd6a', 'ACAd6b',
        'ACAv1', 'ACAv2/3', 'ACAv5', 'ACAv6a', 'ACAv6b'
    ],
    'SS': [
        'SSp-bfd1', 'SSp-bfd2/3', 'SSp-bfd4', 'SSp-bfd5', 'SSp-bfd6a', 'SSp-bfd6b',
        'SSp-ll1', 'SSp-ll2/3', 'SSp-ll4', 'SSp-ll5', 'SSp-ll6a', 'SSp-ll6b',
        'SSp-tr1', 'SSp-tr2/3', 'SSp-tr4', 'SSp-tr5', 'SSp-tr6a', 'SSp-tr6b'
    ],
    'VIS': [
        'VISp1', 'VISp2/3', 'VISp4', 'VISp5', 'VISp6a', 'VISp6b',
        'VISam1', 'VISam2/3', 'VISam4', 'VISam5', 'VISam6a', 'VISam6b',
        'VISpm1', 'VISpm2/3', 'VISpm4', 'VISpm5', 'VISpm6a', 'VISpm6b',
        'VISrl1', 'VISrl2/3', 'VISrl4', 'VISrl5', 'VISrl6a', 'VISrl6b',
    ],
    'MO-TH': [
        'AV', 'CL', 'MD', 'PO', 'RT', 'VAL', 'VPL', 'VPM', 'VM'
    ],
}

area_colors = {
    'MO': 'blue',
    'ACA': 'deepskyblue',
    'SS': 'blueviolet',
    'VIS': 'green',
    'MO-TH': 'steelblue',
}

In [8]:
state_colors = {
    'awake': (120/255, 156/255, 74/255),
    'anesthetized': (130/255, 122/255, 163/255),
    'recovery': (93/255, 167/255, 229/255)
}

## Functions

### From Simone

In [9]:
def AutocleanLFP(InputData):
    PreviousIndexMax = -1
    WhileOn = 1
    while WhileOn:
        if InputData.ndim == 3:
            DataEvoked = np.mean(InputData, axis=2)
        else:
            DataEvoked = InputData

        AbsDiff = np.abs(np.diff(DataEvoked[:,:],2))
        avg = (AbsDiff[:,2:] + AbsDiff[:,0:-2]) / 2
        DevianceMatrix = (AbsDiff[:,1:-1] - avg)
        DevianceVector = np.mean(abs(DevianceMatrix), axis=0)

        StdTh = np.std(DevianceVector)*7
        index_max = max(range(len(DevianceVector)), key=DevianceVector.__getitem__)
        
        if not (index_max == PreviousIndexMax):
            PreviousIndexMax = index_max
            if DevianceVector[index_max] > StdTh:
                if InputData.ndim == 3:
                    InputData[:,index_max+2,:] = (InputData[:,index_max+2+1,:] + InputData[:,index_max+2-1,:]) / 2
                else:
                    InputData[:,index_max+2] = (InputData[:,index_max+2+1] + InputData[:,index_max+2-1]) / 2
#                 print(DevianceVector[index_max] / StdTh)
            else:
                WhileOn = 0
        else:
            WhileOn = 0
    return InputData

def moving_average_2dim(x, w):
    return np.apply_along_axis(lambda m: np.convolve(m, np.ones(w), 'valid'), axis=0, arr=x) / w

In [10]:
def computeCSD(LFPdata, LFPts, baseline_range=[-2, -0.01]):
    BLinds = [find_nearest_ind(LFPts, baseline_range[0]), find_nearest_ind(LFPts, baseline_range[1])]
    tempdata = LFPdata
    
    ## Create buffer above and below the target area's chs for spatial smoothing ##
    ## Do this for the whole probe, not for each area ##
    tmpBoundaryStart = tempdata[:,0,:]
    BoundaryStart = np.repeat(tmpBoundaryStart[:,np.newaxis,:], 14, axis=1)
    tmpBoundaryEnd = tempdata[:,-1,:]
    BoundaryEnd = np.repeat(tmpBoundaryEnd[:,np.newaxis,:], 14, axis=1)
    tempdata = np.concatenate((BoundaryStart, tempdata, BoundaryEnd), axis=1)
    
    ## Baseline correct the LFP ##
    tmpBaselineCorr = np.mean(tempdata[BLinds[0]:BLinds[1],:,:], axis=0)
    tempdata = tempdata - tmpBaselineCorr[np.newaxis,:,:]
    
    ## Create buffer before and after the epoch window for temporal smoothing ##
    tmpBoundaryStart = tempdata[0,:,:]
    BoundaryStart = np.repeat(tmpBoundaryStart[np.newaxis,:,:], 2, axis=0)
    tmpBoundaryEnd = tempdata[-1,:,:]
    BoundaryEnd = np.repeat(tmpBoundaryEnd[np.newaxis,:,:], 1, axis=0)
    tempdata = np.concatenate((BoundaryStart, tempdata, BoundaryEnd), axis=0)
    
    ## Remove bad LFP chs and interpolate data ##
    tempdata = AutocleanLFP(tempdata)
    
    ## Perform spatial and temporal smoothing ##
    ## This should output an array with original LFP dimensions + 2 extra "chs" ##
    CSDdata = np.zeros((LFPdata.shape[0], LFPdata.shape[1]+2, LFPdata.shape[2]))
    for iTrial in range(0, tempdata.shape[2]):
        tmpMatrix = tempdata[:,:,iTrial]
        tmpMatrix = moving_average_2dim(np.transpose(tmpMatrix), 24) # space, corrects for time mismatch due to multiplexing
        tmpMatrix = moving_average_2dim(tmpMatrix, 4) # space, corrects for non-linear ch arrangement
        tmpMatrix = moving_average_2dim(np.transpose(tmpMatrix), 4) # temporal smoothing
        CSDdata[:,:,iTrial] = tmpMatrix
    del tempdata
        
    ## 2nd derivative across chs ##
    CSDdata = np.diff(CSDdata, n=2, axis=1)
    
    ## Another baseline correction ##
    tmpBaselineCorr = np.mean(CSDdata[BLinds[0]:BLinds[1],:,:], axis=0)
    CSDdata = CSDdata - tmpBaselineCorr[np.newaxis,:,:]
    
    return CSDdata

## Load subjects from file

In [11]:
with open(r'C:\Users\lesliec\OneDrive - Allen Institute\data\all_EEG_subjects.json') as subjects_file:
    multi_sub_dict = json.load(subjects_file)

## Load EEG_exp and gather LFP traces

In [12]:
overwrite_existing_files = True

before_event = 2. # s, look at 2 s pre-stim
after_event = 2. # s, look at 2 s post-stim

apply_mask = True
subsampling_factor = 5

In [13]:
for mouse_num, rec_folder in multi_sub_dict.items():
    print('{}'.format(mouse_num))
    exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
    exp_tag = exp.experiment_folder[exp.experiment_folder.find('mouse')+12:exp.experiment_folder.find(str(exp.date.year))-1]
    print(' {}'.format(exp_tag))
    
    probe_list = [x.replace('_sorted', '') for x in exp.experiment_data if 'probe' in x]
    if len(probe_list) == 0:
        print(' This experiment has no probe data, not making LFP/CSD files.\n')
        continue
    
    ## Set file names ##
    evoked_folder = os.path.join(exp.data_folder, 'evoked_data')
    if not os.path.exists(evoked_folder):
        os.mkdir(evoked_folder)
    LFPinfo_filename = os.path.join(evoked_folder, 'evokedLFPinfo.json')
    LFPtime_filename = os.path.join(evoked_folder, 'evokedLFPtimestamps.npy')
    if os.path.exists(LFPinfo_filename) and not overwrite_existing_files:
        print(' LFP traces already exist, not overwriting files.\n')
        continue
    
    ## Load stim log ##
    stim_log = pd.read_csv(exp.stimulus_log_file)
    stim_log = stim_log.astype({'parameter': str})
    all_event_times = stim_log['onset'].values
    
    ## Get probe info ##
    print(' Getting probe info...')
    probe_LFP_data = {}
    probe_locs = np.ones((len(probe_list)), dtype=bool)
    for pbi, probei in enumerate(probe_list):
        ## Load probe_info.json ##
        with open(exp.ephys_params[probei]['probe_info']) as data_file:
            data = json.load(data_file)
        if 'area_ch' in data.keys():
            probe_LFP_data[probei] = {}
            ## Get info ##
            npx_allch = np.array(data['channel']) # this is an array from 0 to 384
            surface_ch = int(data['surface_channel']) # the electrode we said was at the brain surface
            air_ch = int(data['air_channel']) # the electrode at the ACSF/air border
            allch_z = np.array(data['vertical_pos']) # vertical pos of each elec (um), rel to the tip
            ref_mask = np.array(data['mask']) # contains a False for Npx reference channels and "bad chs"
            ## Get all ch depths ##
            npx_chs = np.array([x for x in npx_allch if ref_mask[x] and x <= surface_ch])
            probe_LFP_data[probei]['ch_depths'] = allch_z[surface_ch] - allch_z
            probe_LFP_data[probei]['all_chs'] = npx_allch
            ## Get saline chs for re-ref ##
            if air_ch - 10 > surface_ch:
                probe_LFP_data[probei]['saline_chs'] = np.arange(air_ch - 10, air_ch)
            else:
                probe_LFP_data[probei]['saline_chs'] = np.arange(surface_ch + 1, air_ch)
            ## Save info ##
            probe_LFP_data[probei]['ch_areas'] = np.array(data['area_ch'])
            probe_LFP_data[probei]['pop_chs'] = {key: [] for key in list(areas_of_interest.keys())}
            probe_LFP_data[probei]['pop_ch_depths'] = {key: [] for key in list(areas_of_interest.keys())}
            probe_LFP_data[probei]['pop_ch_areas'] = {key: [] for key in list(areas_of_interest.keys())}
            probe_LFP_data[probei]['pop_ch_mask'] = {key: [] for key in list(areas_of_interest.keys())}
            for chind in npx_allch: ## Now this will look at all chs, not excluding ACSF or bad chs ##
                parent_region = [
                    k for k in list(areas_of_interest.keys()) if probe_LFP_data[probei]['ch_areas'][chind] in areas_of_interest[k]
                ]
                if len(parent_region) == 1:
                    probe_LFP_data[probei]['pop_chs'][parent_region[0]].append(chind)
                    probe_LFP_data[probei]['pop_ch_depths'][parent_region[0]].append(probe_LFP_data[probei]['ch_depths'][chind])
                    probe_LFP_data[probei]['pop_ch_areas'][parent_region[0]].append(probe_LFP_data[probei]['ch_areas'][chind])
                    probe_LFP_data[probei]['pop_ch_mask'][parent_region[0]].append(ref_mask[chind])
        else:
            print('  {} does not have area assignments, not processing.'.format(probei))
            probe_locs[pbi] = False
            
    if probe_locs.any():
        pass
    else:
        print(' NO area assignments for any probes, not analyzing.\n')
        continue
            
    ## Extract LFP traces for each probe ##
    print(' Extracting traces...')
    pop_LFP = {key: [] for key in list(areas_of_interest.keys())}
    pop_CSD = {key: [] for key in list(areas_of_interest.keys())}
    for probei, LFP_data in probe_LFP_data.items():
        ## Memmap LFP data ##
        lfp_ts = np.load(exp.ephys_params[probei]['lfp_timestamps'])
        lfp_mm = np.memmap(
            exp.ephys_params[probei]['lfp_continuous'], dtype='int16',
            shape=(lfp_ts.size, exp.ephys_params[probei]['num_chs']), mode='r')
        ## Mask artifact ##
        if apply_mask:
            mask_samples = int(0.002 * exp.ephys_params[probei]['lfp_sample_rate'])
            lfpdata = lfp_mm.copy()
            for etime in stim_log.loc[stim_log['stim_type'] == 'biphasic', 'onset'].to_numpy():
                val = find_nearest_ind(lfp_ts, etime)
                lfpdata[val:val+mask_samples, :] = lfpdata[val:val-mask_samples:-1, :]
        else:
            lfpdata = lfp_mm.copy()
        del lfp_mm
        ## Downsample timestamps and data ##
        lfp_ts = lfp_ts[::subsampling_factor]
        new_samp_rate = exp.ephys_params[probei]['lfp_sample_rate'] / subsampling_factor
        lfpdata = subsample_lfp(lfpdata, LFP_data['all_chs'], subsampling_factor)
        ## Remove DC offset ##
        lfpdata = remove_lfp_offset(lfpdata, new_samp_rate, 0.1, 1)
        ## Subtract median of ACSF chs ##
        clean_lfp = np.zeros(lfpdata.shape, dtype='int16')
        saline_ref = np.median(lfpdata[:, LFP_data['saline_chs']], axis=1)
        for chi in LFP_data['all_chs']:
            tmp = lfpdata[:, chi] - saline_ref
            clean_lfp[:, chi] = tmp.astype('int16')
        del lfpdata
        ## Get evoked LFP traces ##
        lfp_event_traces, lfp_event_ts = get_evoked_traces(
            clean_lfp, lfp_ts, stim_log['onset'].values, before_event, after_event, new_samp_rate)
        lfp_event_traces = lfp_event_traces * exp.ephys_params[probei]['bit_volts']
        del lfp_ts, clean_lfp
        
        ## Analyze CSD ##
        csd_event_traces = computeCSD(lfp_event_traces, lfp_event_ts)
        ## Store LFP and CSD event traces ##
        for region in LFP_data['pop_chs'].keys():
            pop_chs = LFP_data['pop_chs'][region]
            if len(pop_chs) == 0:
                continue
            pop_LFP[region].append(lfp_event_traces[:, pop_chs, :])
            pop_CSD[region].append(csd_event_traces[:, pop_chs, :])
        del lfp_event_traces, csd_event_traces
        
    ## Reorganize data ##
    pop_ch_depths = {key: [] for key in list(areas_of_interest.keys())}
    pop_ch_areas = {key: [] for key in list(areas_of_interest.keys())}
    pop_ch_mask = {key: [] for key in list(areas_of_interest.keys())}
    for probei in probe_LFP_data.keys():
        for region in probe_LFP_data[probei]['pop_ch_depths'].keys():
            pop_ch_depths[region].append(probe_LFP_data[probei]['pop_ch_depths'][region])
            pop_ch_areas[region].append(probe_LFP_data[probei]['pop_ch_areas'][region])
            pop_ch_mask[region].append(probe_LFP_data[probei]['pop_ch_mask'][region])

    ## Save the data ##
    print(' Saving traces...')
    LFPtraces_info = {}
    for region in pop_LFP.keys():
        LFP_filename = os.path.join(evoked_folder, region + '_evokedLFPtraces.npy')
        CSD_filename = os.path.join(evoked_folder, region + '_evokedCSDtraces.npy')
        if len(pop_LFP[region]) == 0:
            print('  No chs were found for {}, not saving data.'.format(region))
            continue
        LFPtraces_info[region] = {}

        reg_ch_depths = np.concatenate(pop_ch_depths[region])
        reg_ch_areas = np.concatenate(pop_ch_areas[region])
        reg_ch_mask = np.concatenate(pop_ch_mask[region]).astype(bool)
        LFP_traces = np.concatenate(pop_LFP[region], axis=1)
        CSD_traces = np.concatenate(pop_CSD[region], axis=1)

        CH_DEPTH_SORT = np.argsort(reg_ch_depths)
        LFPtraces_info[region]['ch_depths'] = reg_ch_depths[CH_DEPTH_SORT].tolist()
        LFPtraces_info[region]['ch_areas'] = reg_ch_areas[CH_DEPTH_SORT].tolist()
        LFPtraces_info[region]['ch_mask'] = reg_ch_mask[CH_DEPTH_SORT].tolist()

#         print('  Saving {}.'.format(LFP_filename))
        np.save(LFP_filename, LFP_traces[:, CH_DEPTH_SORT, :], allow_pickle=False)
#         print('  Saving {}.'.format(CSD_filename))
        np.save(CSD_filename, CSD_traces[:, CH_DEPTH_SORT, :], allow_pickle=False)

    LFPtraces_info['sample_rate'] = new_samp_rate
    
    print('  Saving {}.'.format(LFPinfo_filename))
    with open(LFPinfo_filename, 'w') as outfile:
        json.dump(LFPtraces_info, outfile, indent = 4, separators = (',', ': '))
    print('  Saving {}.'.format(LFPtime_filename))
    np.save(LFPtime_filename, lfp_event_ts, allow_pickle=False)
    print('')

521885
Experiment type: electrical stimulation
 estim1
 This experiment has no probe data, not making LFP/CSD files.

521886
Experiment type: electrical stimulation
 estim1
 This experiment has no probe data, not making LFP/CSD files.

521887
Experiment type: electrical stimulation
 estim1
 This experiment has no probe data, not making LFP/CSD files.

543393
Experiment type: electrical stimulation
 estim1
 Getting probe info...
  probeB does not have area assignments, not processing.
 NO area assignments for any probes, not analyzing.

543394
Experiment type: electrical stimulation
 estim1
 Getting probe info...
  probeB does not have area assignments, not processing.
 NO area assignments for any probes, not analyzing.

543395
Experiment type: electrical stimulation
 estim1
 Getting probe info...
  probeB does not have area assignments, not processing.
 NO area assignments for any probes, not analyzing.

543396
Experiment type: electrical and sensory stimulation
 estim_vis1
 Getting pr

Experiment type: electrical and sensory stimulation
 estim_vis
 Getting probe info...
 Extracting traces...
 Saving traces...
  Saving F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1\evoked_data\evokedLFPinfo.json.
  Saving F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1\evoked_data\evokedLFPtimestamps.npy.

582387
Experiment type: electrical and sensory stimulation
 estim_vis
 Getting probe info...
 Extracting traces...
 Saving traces...
  Saving F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1\evoked_data\evokedLFPinfo.json.
  Saving F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1\evoked_data\evokedLFPtimestamps.npy.

586466
Experiment type: electrical and sensory stimulation
 estim_vis
 Getting probe info...
  probeC does not have area assignments, not processing.
 Extracting traces...
 Saving traces...
  No chs were found for VIS, not saving data.
  No chs were found for M

## Test on single subject

### Define data path

In [11]:
rec_folder = r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1'
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
    
exp_tag = exp.experiment_folder[exp.experiment_folder.find('mouse')+12:exp.experiment_folder.find(str(exp.date.year))-1]
print(exp_tag)

Experiment type: electrical stimulation
estim2


In [12]:
# Make a folder in my OneDrive to save plots
plotsavedir = os.path.join(r'C:\Users\lesliec\OneDrive - Allen Institute\data\plots', 'mouse' + exp.mouse)
if not os.path.exists(plotsavedir):
    os.mkdir(plotsavedir)

#### Load stim table and speed signal

In [13]:
stim_log = pd.read_csv(exp.stimulus_log_file)
stim_log = stim_log.astype({'parameter': str})
stim_log.head()

,stim_type,parameter,duration,onset,offset,sweep,good,mean_speed,resting_trial
0,biphasic,40,0.0004,319.16422,319.16462,0,True,6.097344,False
1,biphasic,20,0.0004,322.69724,322.69764,0,True,1.321218,False
2,biphasic,40,0.0004,326.75887,326.75927,0,True,0.387253,False
3,biphasic,60,0.0004,331.14116,331.14156,0,True,0.000000,True
4,biphasic,60,0.0004,335.31077,335.31117,0,True,21.686193,False


#### Load unit data

In [14]:
probe_list = [x.replace('_sorted', '') for x in exp.experiment_data if 'probe' in x]
print(probe_list)

['probeB', 'probeC', 'probeF']


In [15]:
probe_LFP_data = {}
for probei in probe_list:
    print(probei)
    probe_LFP_data[probei] = {}
    ## Load probe_info.json ##
    with open(exp.ephys_params[probei]['probe_info']) as data_file:
        data = json.load(data_file)
    npx_allch = np.array(data['channel']) # this is an array from 0 to 384
    surface_ch = int(data['surface_channel']) # the electrode we said was at the brain surface
    air_ch = int(data['air_channel']) # the electrode at the ACSF/air border
    allch_z = np.array(data['vertical_pos']) # vertical pos of each elec (um), rel to the tip (ch 0 is 20 um from tip)
    ref_mask = np.array(data['mask']) # contains a False for Npx reference channels and "bad chs"
    print(' surface channel: {:d}'.format(surface_ch))
    
    ## Get all ch depths ##
    npx_chs = np.array([x for x in npx_allch if ref_mask[x] and x <= surface_ch])
    probe_LFP_data[probei]['ch_depths'] = allch_z[surface_ch] - allch_z
    probe_LFP_data[probei]['all_chs'] = npx_allch
    
    ## Get saline chs for re-ref ##
    if air_ch - 10 > surface_ch:
        probe_LFP_data[probei]['saline_chs'] = np.arange(air_ch - 10, air_ch)
    else:
        probe_LFP_data[probei]['saline_chs'] = np.arange(surface_ch + 1, air_ch)
    if 'area_ch' in data.keys():
        probe_LFP_data[probei]['ch_areas'] = np.array(data['area_ch'])
        probe_LFP_data[probei]['pop_chs'] = {key: [] for key in list(areas_of_interest.keys())}
        probe_LFP_data[probei]['pop_ch_depths'] = {key: [] for key in list(areas_of_interest.keys())}
        probe_LFP_data[probei]['pop_ch_areas'] = {key: [] for key in list(areas_of_interest.keys())}
        probe_LFP_data[probei]['pop_ch_mask'] = {key: [] for key in list(areas_of_interest.keys())}
        for chind in npx_allch: ## Now this will look at all chs, not excluding ACSF or bad chs ##
            parent_region = [
                k for k in list(areas_of_interest.keys()) if probe_LFP_data[probei]['ch_areas'][chind] in areas_of_interest[k]
            ]
            if len(parent_region) == 1:
                probe_LFP_data[probei]['pop_chs'][parent_region[0]].append(chind)
                probe_LFP_data[probei]['pop_ch_depths'][parent_region[0]].append(probe_LFP_data[probei]['ch_depths'][chind])
                probe_LFP_data[probei]['pop_ch_areas'][parent_region[0]].append(probe_LFP_data[probei]['ch_areas'][chind])
                probe_LFP_data[probei]['pop_ch_mask'][parent_region[0]].append(ref_mask[chind])

probeB
 surface channel: 340
probeC
 surface channel: 350
probeF
 surface channel: 285


In [20]:
bugs = np.concatenate([probe_LFP_data['probeB']['pop_ch_mask']['SS']])
type(bugs)

numpy.ndarray

In [26]:
bellgs = bugs[::-1]

In [34]:
bool(bugs[45])

False

In [27]:
bellgs

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [30]:
beegs = bugs.astype(bool).tolist()
print(beegs)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [31]:
type(beegs[0])

bool

### Get evoked LFP

In [ ]:
before_event = 2. # s, look at 2 s pre-stim
after_event = 2. # s, look at 2 s post-stim

apply_mask = True
subsampling_factor = 5 # default = 2, can set as int value or None

In [ ]:
probei = 'probeB'

## Memmap LFP data ##
start = time.time()
lfp_ts = np.load(exp.ephys_params[probei]['lfp_timestamps'])
lfp_mm = np.memmap(
    exp.ephys_params[probei]['lfp_continuous'], dtype='int16',
    shape=(lfp_ts.size, exp.ephys_params[probei]['num_chs']), mode='r')
end = time.time()
print('time to memmap cont data: {:.2f} min'.format((end-start)/60))

## Mask artifact ## loads to memory
if apply_mask:
    start = time.time()
    mask_samples = int(0.002 * exp.ephys_params[probei]['lfp_sample_rate'])
    lfpdata = lfp_mm.copy()
    for etime in stim_log.loc[stim_log['stim_type'] == 'biphasic', 'onset'].to_numpy():
        val = find_nearest_ind(lfp_ts, etime)
        lfpdata[val:val+mask_samples, :] = lfpdata[val:val-mask_samples:-1, :]
    end = time.time()
    print('time to mask artifact: {:.2f} min'.format((end-start)/60))
else:
    lfpdata = lfp_mm.copy()
del lfp_mm

## Downsample timestamps and data ## reduces memory requirement if downsampled
if subsampling_factor:
    start = time.time()
    lfp_ts = lfp_ts[::subsampling_factor]
    new_samp_rate = exp.ephys_params[probei]['lfp_sample_rate'] / subsampling_factor
    lfpdata = subsample_lfp(lfpdata, probe_LFP_data[probei]['all_chs'], subsampling_factor)
    end = time.time()
    print('time to downsample: {:.2f} min'.format((end-start)/60))
else:
    new_samp_rate = exp.ephys_params[probei]['lfp_sample_rate']

## Remove DC offset ##
start = time.time()
lfpdata = remove_lfp_offset(lfpdata, new_samp_rate, 0.1, 1)
end = time.time()
print('time to highpass filter: {:.2f} min'.format((end-start)/60))

## Subtract median of ACSF chs ##
start = time.time()
clean_lfp = np.zeros(lfpdata.shape, dtype='int16')
saline_ref = np.median(lfpdata[:, probe_LFP_data[probei]['saline_chs']], axis=1)
for chi in probe_LFP_data[probei]['all_chs']:
    tmp = lfpdata[:, chi] - saline_ref
    clean_lfp[:, chi] = tmp.astype('int16')
del lfpdata
end = time.time()
print('time to re-reference: {:.2f} min'.format((end-start)/60))

## Get evoked LFP traces ## increases amount of data in memory
start = time.time()
lfp_event_traces, lfp_event_ts = get_evoked_traces(
    clean_lfp, lfp_ts, stim_log['onset'].values, before_event, after_event, new_samp_rate)
lfp_event_traces = lfp_event_traces * exp.ephys_params[probei]['bit_volts']
del lfp_ts, clean_lfp
end = time.time()
print('time to epoch LFP: {:.2f} min'.format((end-start)/60))

In [ ]:
lfp_event_traces.shape

In [ ]:
baseline_range = [-2, -0.01]
tempdata = lfp_event_traces
LFPts = lfp_event_ts

BLinds = [find_nearest_ind(LFPts, baseline_range[0]), find_nearest_ind(LFPts, baseline_range[1])] 

In [ ]:
## Create buffer above and below the target area's chs for spatial smoothing ##
## Do this for the whole probe, not for each area ##
tmpBoundaryStart = tempdata[:,0,:]
BoundaryStart = np.repeat(tmpBoundaryStart[:,np.newaxis,:], 14, axis=1)
tmpBoundaryEnd = tempdata[:,-1,:]
BoundaryEnd = np.repeat(tmpBoundaryEnd[:,np.newaxis,:], 14, axis=1)
tempdata = np.concatenate((BoundaryStart, tempdata, BoundaryEnd), axis=1)

In [ ]:
lfp_event_traces.shape

In [ ]:
tempdata.shape

In [ ]:
## Baseline correct the LFP ##
tmpBaselineCorr = np.mean(tempdata[BLinds[0]:BLinds[1],:,:], axis=0)
tempdata = tempdata - tmpBaselineCorr[np.newaxis,:,:]

In [ ]:
## Create buffer before and after the epoch window for temporal smoothing ##
tmpBoundaryStart = tempdata[0,:,:]
BoundaryStart = np.repeat(tmpBoundaryStart[np.newaxis,:,:], 2, axis=0)
tmpBoundaryEnd = tempdata[-1,:,:]
BoundaryEnd = np.repeat(tmpBoundaryEnd[np.newaxis,:,:], 1, axis=0)
tempdata = np.concatenate((BoundaryStart, tempdata, BoundaryEnd), axis=0)

In [ ]:
lfp_event_traces.shape

In [ ]:
tempdata.shape

In [ ]:
start = time.time()
## Remove bad LFP chs and interpolate data ##
tempdata = AutocleanLFP(tempdata)
end = time.time()
print('time to auto-clean LFP: {:.2f} min'.format((end-start)/60))

In [ ]:
start = time.time()
## Perform spatial and temporal smoothing ##
## This should output an array with original LFP dimensions ##
smoothdata = np.zeros((lfp_event_traces.shape[0], lfp_event_traces.shape[1]+2, lfp_event_traces.shape[2]))
for iTrial in range(0, tempdata.shape[2]):
    tmpMatrix = tempdata[:,:,iTrial]
    tmpMatrix = moving_average_2dim(np.transpose(tmpMatrix), 24) # space, corrects for time mismatch due to multiplexing
    tmpMatrix = moving_average_2dim(tmpMatrix, 4) # space, corrects for non-linear ch arrangement
    tmpMatrix = moving_average_2dim(np.transpose(tmpMatrix), 4) # temporal smoothing
    smoothdata[:,:,iTrial] = tmpMatrix
del tempdata
end = time.time()
print('time to smooth LFP in space and time: {:.2f} min'.format((end-start)/60))

In [ ]:
print(lfp_event_traces.shape)
print(smoothdata.shape)
print(tmpMatrix.shape)

In [ ]:
## 2nd derivative across chs ##
CSDdata = np.diff(smoothdata, n=2, axis=1)
del smoothdata
## Another baseline correction ##
tmpBaselineCorr = np.mean(CSDdata[BLinds[0]:BLinds[1],:,:], axis=0)
CSDdata = CSDdata - tmpBaselineCorr[np.newaxis,:,:]

In [ ]:
CSDdata.shape

### All probes

In [ ]:
pop_LFP = {key: [] for key in list(areas_of_interest.keys())}
raw_traces = {}
for probei, LFP_data in probe_LFP_data.items():
    ## Memmap LFP data ##
    print(probei)
    start = time.time()
    lfp_ts = np.load(exp.ephys_params[probei]['lfp_timestamps'])
    lfp_mm = np.memmap(
        exp.ephys_params[probei]['lfp_continuous'], dtype='int16',
        shape=(lfp_ts.size, exp.ephys_params[probei]['num_chs']), mode='r')
    
    ## Mask artifact ##
    if apply_mask:
        mask_samples = int(0.002 * exp.ephys_params[probei]['lfp_sample_rate'])
        lfpdata = lfp_mm.copy()
        for etime in stim_log.loc[stim_log['stim_type'] == 'biphasic', 'onset'].to_numpy():
            val = find_nearest_ind(lfp_ts, etime)
            lfpdata[val:val+mask_samples, :] = lfpdata[val:val-mask_samples:-1, :]
    else:
        lfpdata = lfp_mm.copy()
    del lfp_mm
    
    ## Downsample timestamps and data ##
    lfp_ts = lfp_ts[::subsampling_factor]
    new_samp_rate = exp.ephys_params[probei]['lfp_sample_rate'] / subsampling_factor
    lfpdata = subsample_lfp(lfpdata, LFP_data['all_chs'], subsampling_factor)
    
    ## Remove DC offset ##
    lfpdata = remove_lfp_offset(lfpdata, new_samp_rate, 0.1, 1)
    
    ## Subtract median of ACSF chs ##
    clean_lfp = np.zeros(lfpdata.shape, dtype='int16')
    saline_ref = np.median(lfpdata[:, LFP_data['saline_chs']], axis=1)
    for chi in LFP_data['all_chs']:
        tmp = lfpdata[:, chi] - saline_ref
        clean_lfp[:, chi] = tmp.astype('int16')
    del lfpdata
    
    ## Get evoked LFP traces ##
    lfp_event_traces, lfp_event_ts = get_evoked_traces(
        clean_lfp, lfp_ts, stim_log['onset'].values, before_event, after_event, new_samp_rate)
    lfp_event_traces = lfp_event_traces * exp.ephys_params[probei]['bit_volts']
    
    raw_traces[probei] = [lfp_event_ts, lfp_event_traces]
    for region in LFP_data['pop_chs'].keys():
        pop_chs = LFP_data['pop_chs'][region]
        if len(pop_chs) == 0:
            continue
        pop_LFP[region].append(lfp_event_traces[:, pop_chs, :])

    del lfp_ts
    del clean_lfp
    del lfp_event_traces
    del lfp_event_ts
    
    end = time.time()
    print(' Time to pre-process and epoch: {:.2f} min'.format((end-start)/60))

In [ ]:
pop_ch_depths = {key: [] for key in list(areas_of_interest.keys())}
pop_ch_areas = {key: [] for key in list(areas_of_interest.keys())}
for probei in probe_LFP_data.keys():
    for region in probe_LFP_data[probei]['pop_ch_depths'].keys():
        pop_ch_depths[region].append(probe_LFP_data[probei]['pop_ch_depths'][region])
        pop_ch_areas[region].append(probe_LFP_data[probei]['ch_areas'][probe_LFP_data[probei]['pop_chs'][region]])

### Plot population LFP traces

In [ ]:
triali = 11
vlevel = 5000

fig, axs = plt.subplots(len(areas_of_interest), 1, figsize=(8,8), sharex=True, constrained_layout=True)
timex = raw_traces['probeB'][0]
for rowi, region in enumerate(areas_of_interest):
    if len(pop_LFP[region]) == 0:
        continue
        
    ## Plot LFP ##
    LFPtraces = np.concatenate(pop_LFP[region], axis=1)
    ch_depth_sort = np.squeeze(np.argsort(np.concatenate(pop_ch_depths[region], axis=0)))[::-1]
    axs[rowi].imshow(
        LFPtraces[:, ch_depth_sort, triali].T, cmap='bwr', interpolation='none', aspect='auto', origin='lower',
        vmin=-vlevel, vmax=vlevel, extent=[timex[0], timex[-1], 0, ch_depth_sort.shape[0]]
    )
    
    axs[rowi].set_ylabel('{} LFP\ndeep <---> sup.'.format(region))
    
axs[rowi].set_xlabel('Time from stim onset (s)')    
plt.suptitle('Evoked LFP traces for a single trial\n{}: trial {:d}'.format(exp.mouse, triali), fontsize=11)
## Save ##
figname = 'allPOP_LFPtraces_{}_trial{:d}-resting.png'.format(exp.mouse, triali)
# fig.savefig(os.path.join(plotsavedir, figname), transparent=False)

## Save the data

In [ ]:
LFPtraces_info = {}
for region in pop_LFP.keys():
    print(region)
    start = time.time()
    LFP_filename = os.path.join(exp.data_folder, region + '_evokedLFPtraces.npy')
    if len(pop_LFP[region]) == 0:
        print(' No chs were found for this region, not saving data.\n')
        continue
    LFPtraces_info[region] = {}
    
    reg_ch_depths = np.concatenate(pop_ch_depths[region])
    reg_ch_areas = np.concatenate(pop_ch_areas[region])
    LFP_traces = np.concatenate(pop_LFP[region], axis=1)
    
    CH_DEPTH_SORT = np.argsort(reg_ch_depths)
    LFPtraces_info[region]['ch_depths'] = reg_ch_depths[CH_DEPTH_SORT].tolist()
    LFPtraces_info[region]['ch_areas'] = reg_ch_areas[CH_DEPTH_SORT].tolist()
    
    print(' Saving {}.'.format(LFP_filename))
#     np.save(LFP_filename, LFP_traces[:, CH_DEPTH_SORT, :], allow_pickle=False)
    end = time.time()
    print(' Time to save: {:.2f} s'.format(end-start))
    print('')
    
LFPinfo_filename = os.path.join(exp.data_folder, 'evokedLFPinfo.json')
with open(LFPinfo_filename, 'w') as outfile:
    json.dump(LFPtraces_info, outfile, indent = 4, separators = (',', ': '))
    
LFPtime_filename = os.path.join(exp.data_folder, 'evokedLFPtimestamps.npy')
np.save(LFPtime_filename, timex, allow_pickle=False)

### Original functions from Simone